In [189]:
import numpy as np
from qutip import *
import plotly.graph_objects as go
import os
from qutip import *
from tqdm import tqdm 

In [190]:
# --------------------------
# Platform scaling (pick one)
# --------------------------
PLATFORM = "ibm"   # "ibm" or "quera"

# Platform temperature for thermal factors
def platform_params(platform="ibm"):
    if platform.lower() == "ibm":
        # IBM-style transmon (rad/s, seconds)
        omega_max = 2*np.pi*12.5e6      # 40 ns pi pulse -> 12.5 MHz
        # T_max     = 0.32e-6             # ~8*pi/omega_max; use 1e-6..2e-6 for very adiabatic
        T_max     = 2e-6             # ~8*pi/omega_max; use 1e-6..2e-6 for very adiabatic
        Ep        = 2*np.pi*100e6       # penalty ~100 MHz
        time_unit_label = "time [µs]"
        energy_unit_label = "energy [MHz]"  # plots will convert rad/s -> MHz
        to_time_units = 1e6             # s -> µs
        to_freq_units = 1/(2*np.pi*1e6) # rad/s -> MHz
        lambda_2 = 1e4
        T_phys_K = 0.015 # K

        # Bath temperatures (Kelvin) typical for dilution fridges
        temperatures = np.array([0.008, 0.010, 0.012, 0.015, 0.020, 0.030, 0.050, 0.100], dtype=float)
        # Dimensionless Ep/(k_B T) values for EGP scaling
        beta_vec     = np.array([0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0], dtype=float)
        # Penalty sweep (rad/s): 2π × {5,10,20,50,100,200} MHz
        Ep_vec       = 2*np.pi*np.array([10e6, 20e6, 50e6, 100e6, 200e6, 500e6, 1000e6], dtype=float) # rad/s
        # Ep_vec = 2*np.pi*np.linspace(5, 500,20) * 1e6

    elif platform.lower() == "quera":
        # QuEra Aquila (rad/µs, microseconds)
        omega_max = 10.0                # <= 15.8 rad/µs
        # T_max     = 1.9                 # µs
        T_max     = 3                 # µs
        Ep        = 30.0                # rad/µs (>> omega_max)
        time_unit_label = "time [µs]"
        energy_unit_label = "energy [MHz]"  # convert rad/µs -> MHz
        to_time_units = 1.0             # µs -> µs
        to_freq_units = 1/(2*np.pi)     # rad/µs -> MHz
        lambda_2 = 5e-3
        T_phys_K = 300 # K

        # Room-temp EM environment relevant for BBR (Kelvin)
        temperatures = np.array([260.0, 273.15, 295.0, 300.0, 310.0, 330.0], dtype=float)
        # At room T, Ep/(k_B T) is tiny for MHz-scale Ep → use decade-spaced small betas
        beta_vec     = np.array([1e-9, 3e-9, 1e-8, 3e-8, 1e-7, 3e-7, 1e-6], dtype=float)
        # Penalty sweep (rad/µs): {5,10,20,30,40,60}
        Ep_vec       = np.array([5.0, 10.0, 20.0, 30.0, 40.0, 60.0], dtype=float)

    else:
        raise ValueError("PLATFORM must be 'ibm' or 'quera'")
    return dict(omega_max=omega_max, T_max=T_max, Ep=Ep,
                time_unit_label=time_unit_label,
                energy_unit_label=energy_unit_label,
                to_time_units=to_time_units,
                to_freq_units=to_freq_units,
                temperatures=temperatures,
                Ep_vec=Ep_vec, beta_vec=beta_vec, lambda_2=lambda_2, T_phys_K=T_phys_K)

params = platform_params(PLATFORM)
omega_max = params["omega_max"]
T_max     = params["T_max"]
Ep        = params["Ep"]
Ep_vec    = params["Ep_vec"]
temperatures = params["temperatures"]
beta_vec  = params["beta_vec"]
lambda_2 = params["lambda_2"]
T_phys_K = params["T_phys_K"]

# --------------------------
# Single-qubit primitives
# --------------------------
I = qeye(2)
X = sigmax()
Y = sigmay()
Z = sigmaz()

# --------------------------
# Logical operators (3-qubit repetition, bit-flip code)
# Code space: |0_L>=|000>, |1_L>=|111>
# Stabilizers for X-error protection: Z1Z2 and Z2Z3
# --------------------------
X_L = tensor(X, X, X)
Z_L = tensor(Z, Z, Z)              # acts nontrivially on code; anticommutes with X_L
I_L = tensor(I, I, I)

S1 = tensor(Z, Z, I)               # Z1 Z2
S2 = tensor(I, Z, Z)               # Z2 Z3
S_list = [S1, S2]

# Penalty Hamiltonian: -Ep * (S1 + S2)
Hp = -Ep * sum(S_list, 0*I_L)      # 'start' ensures Qobj sum

# --------------------------
# Basis & logical states (optional, for checks)
# --------------------------
basis_states = [basis(2, 0), basis(2, 1)]
logical_zero = tensor(basis_states[0], basis_states[0], basis_states[0])
logical_one  = tensor(basis_states[1], basis_states[1], basis_states[1])

# --------------------------
# RAP pulses
# --------------------------
def omega_t(t, T_max=T_max, omega_max=omega_max):
    return omega_max * np.sin(np.pi * t / T_max)

def delta_t(t, T_max=T_max, omega_max=omega_max):
    return -omega_max * np.cos(np.pi * t / T_max)

def H(t):
    # Logical RAP drive
    Ht = X_L * omega_t(t) + Z_L * delta_t(t) + Hp
    return Ht.to('csr')

# --------------------------
# Time grid
# --------------------------
def time_list(num_points=51):
    return np.linspace(0, T_max, num_points)

t_list = time_list()

# --------------------------
# Spectrum vs. time with simple eigenstate matching
# --------------------------
def spectrum_track(t_list, n_track=2):
    """
    Track the lowest 'n_track' eigenstates by overlap with t=0 eigenstates.
    Returns: dict {idx: energies_over_time}
    """
    # t=0 diagonalization
    evals0, estates0 = H(0).eigenstates()
    dim = len(evals0)
    energies = {i: [evals0[i]] for i in range(dim)}

    for k in range(1, len(t_list)):
        t = t_list[k]
        evals, estates = H(t).eigenstates()

        # Track the first n_track states by overlap with initial estates0
        used = set()
        for i in range(n_track):
            ref = estates0[i]
            best_j, best_over = 0, -1.0
            for j, psi in enumerate(estates):
                if j in used:
                    continue
                ov = abs(ref.overlap(psi))**2
                if ov > best_over:
                    best_over, best_j = ov, j
            energies[i].append(evals[best_j])
            used.add(best_j)

        # Append the rest by index (no tracking)
        for i in range(n_track, dim):
            energies[i].append(evals[i])

    return energies, params


# =========================
# Additions for open-system
# =========================

# --- Physical constants
kB   = 1.380649e-23        # J/K
hbar = 1.054571817e-34     # J*s

In [191]:
energies, p = spectrum_track(t_list, n_track=2)

# --------------------------
# Plot with Plotly (kept your style; kaleido optional)
# --------------------------

# Pulses
omega_vals = [omega_t(t) for t in t_list]
delta_vals = [delta_t(t) for t in t_list]

# Convert to nice plot units
t_plot   = t_list * p["to_time_units"]
w_plot   = np.array(omega_vals) * p["to_freq_units"]
d_plot   = np.array(delta_vals) * p["to_freq_units"]

os.makedirs('pngs', exist_ok=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t_plot, y=w_plot, mode='lines', name='Ω(t)'))
fig.add_trace(go.Scatter(x=t_plot, y=d_plot, mode='lines', name='Δ(t)', line=dict(dash='dash')))
fig.update_layout(
    title=f"RAP pulses ({PLATFORM.upper()})",
    xaxis_title=p["time_unit_label"],
    yaxis_title="amplitude [MHz]",
    template="simple_white"
)
try:
    fig.write_image('pngs/omega_delta.png', scale=2, width=800, height=500)
except Exception as e:
    pass
fig.show()

# Spectrum figure
fig2 = go.Figure()
colors_main = ['red', 'blue']
for i, e_list in energies.items():
    e_plot = np.array(e_list) * p["to_freq_units"]
    if i < 2:
        fig2.add_trace(go.Scatter(x=t_plot, y=e_plot, mode='lines',
                                  line=dict(width=3, color=colors_main[i%2]),
                                  name=f"tracked state {i}"))
    else:
        fig2.add_trace(go.Scatter(x=t_plot, y=e_plot, mode='lines',
                                  line=dict(width=1, dash='dash'),
                                  name=f"state {i}"))
fig2.update_layout(
    title=f"Instantaneous spectrum vs time ({PLATFORM.upper()})",
    xaxis_title=p["time_unit_label"],
    yaxis_title=p["energy_unit_label"],
    template="simple_white",
    legend_title=""
)
# try:
#     fig2.write_image('pngs/spectrum.png', scale=2, width=900, height=600)
# except Exception as e:
#     pass
fig2.show()


In [154]:
# ---------- Hamiltonians ----------
def H0(t):
    """Bare RAP logical Hamiltonian (no penalty)."""
    Ht = X_L * omega_t(t) + Z_L * delta_t(t)
    return Ht.to('csr')

def make_H_func(Ep_val):
    """Return H(t, args) including the chosen penalty."""
    S_sum = S1 + S2
    def _H(t, args=None):
        Ht = H0(t) - Ep_val * S_sum
        return Ht.to('csr')
    return _H

# ---------- Thermal factor (unit-consistent) ----------
# def bose_einstein_N(omega_pos, T_K, platform=PLATFORM):
#     if T_K <= 0 or omega_pos <= 0:
#         return 0.0
#     kBT_over_hbar_rad_per_s = (kB * T_K) / hbar      # [rad/s]
#     kBT_over_hbar = kBT_over_hbar_rad_per_s * (1e-6 if platform.lower() == "quera" else 1.0)
#     x = omega_pos / kBT_over_hbar
#     x = np.clip(x, 1e-12, 700.0)
#     return 1.0 / (np.exp(x) - 1.0)

def bose_einstein_N(energy_diff, temperature):
    if temperature == 0:
        return 0
    return 1.0 / (np.exp(energy_diff*hbar / (kB * temperature)) - 1)


def spectral_amp(omega_pos, lambda_2):
    return 0.0 if omega_pos <= 0 else np.sqrt(lambda_2)


def spectral_amp(omega_pos, lambda_2, platform=PLATFORM, model='ibm',
                 omega_c=None, omega_ref=None, s=1.0):
    """
    Returns sqrt( |g_ba|^2 ) = sqrt( lambda_2 * f(omega_pos) ).
    - If model='flat' : f(ω)=1 (your current assumption).
    - If model='ohmic': f(ω)=(ω/ω_ref)^s * exp(-ω/ω_c)  [default s=1].
    Units:
      - ibm: ω in rad/s, omega_c in rad/s
      - quera: ω in rad/µs, omega_c in rad/µs
    """
    if omega_pos <= 0:
        return 0.0
    p = platform.lower().strip()
    if model is None:
        # neutral atoms: flat is a reasonable effective model; transmons: ohmic is common
        model = 'flat' if p == 'quera' else 'ohmic'

    # sensible references if user doesn't pass them
    if omega_ref is None:
        omega_ref = max(1e-12, float(params.get('omega_max', omega_pos)))
    if omega_c is None:
        omega_c = (2*np.pi*2e8) if p == 'ibm' else 50.0   # ~200 MHz cutoff (IBM) or ~50 rad/µs (QuEra)

    if model == 'flat':
        f = 1.0
    elif model == 'ohmic':
        f = (omega_pos/omega_ref)**float(s) * np.exp(-omega_pos/omega_c)
    else:
        f = 1.0  # fallback to flat

    return np.sqrt(lambda_2 * f)

# ---------- Collapse-operator generator ----------
def c_ops_gen_thermal(t_eval, H_func, n_qubits, lambda_2, T_K):
    Ht = H_func(t_eval, None)
    evals, evecs = Ht.eigenstates()
    dim = len(evals)

    c_ops = []
    for b in range(2, dim):
        omega = float(evals[b] - evals[0])  # >0
        if abs(omega) < 1e-8:
            continue

        rate_abs = 0.0  # 0 -> b
        rate_em  = 0.0  # b -> 0
        for i in range(n_qubits):
            sigmam_i = tensor([sigmam() if j == i else I for j in range(n_qubits)])
            # complex scalar matrix element <b|sigmam_i|0>
            m_ba = (evecs[b].dag() * sigmam_i * evecs[0])
            mag2 = float(np.abs(m_ba)**2)
            if mag2 < 1e-8:
                continue

            g = spectral_amp(omega, lambda_2)
            N = bose_einstein_N(omega, T_K)
            rate_abs += N       * (g**2) * mag2
            rate_em  += (N + 1) * (g**2) * mag2

        if rate_abs > 1e-8:
            Lop = (evecs[b] * evecs[0].dag()).to('csr')      # |b><0|
            c_ops.append(np.sqrt(rate_abs) * Lop)
        if rate_em > 1e-8:
            Lop = (evecs[0] * evecs[b].dag()).to('csr')      # |0><b|
            c_ops.append(np.sqrt(rate_em) * Lop)
    return c_ops

In [155]:
# ---------- Simulation setup ----------
n_qubits = 3
initial_state = logical_zero

rho0 = logical_zero * logical_zero.dag()
rho1 = logical_one  * logical_one.dag()
e_ops = [rho0, rho1]

# Ideal (unitary) benchmark with Ep=0
H_ideal = make_H_func(Ep_val=0.0)
res_ideal = sesolve(H_ideal, initial_state, t_list, e_ops=e_ops)
ideal_fidelity = res_ideal.expect[1]   # <1_L|rho|1_L>


T_K        = params["T_phys_K"]

def _last_state(result):
    """Robustly fetch last state from a QuTiP result."""
    if hasattr(result, "states") and result.states:
        return result.states[-1]
    if hasattr(result, "state") and (result.state is not None):
        return result.state
    raise RuntimeError("Solver returned no states. Enable store_states=True.")

fidelity_curves = []
for Ep_val in Ep_vec:
    print(f"Ep = {Ep_val}")
    H_fun = make_H_func(Ep_val)

    rho = initial_state * initial_state.dag()
    f_track = [float((rho1*rho).tr().real)]  # include t0

    for k in range(1, len(t_list)):
        t0, t1  = t_list[k-1], t_list[k]
        t_mid   = 0.5*(t0 + t1)
        c_ops   = c_ops_gen_thermal(t_mid, H_fun, n_qubits=n_qubits,
                                    lambda_2=lambda_2, T_K=T_K)

        opts = Options(store_states=True)
        result = mesolve(H_fun, rho, [t0, t1], c_ops=c_ops,
                         e_ops=e_ops, args=None, options=opts)

        rho = _last_state(result)
        f_track.append(result.expect[1][-1])

    fidelity_curves.append(np.array(f_track))

# ---------- Plot ----------
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_list*params["to_time_units"], y=ideal_fidelity,
                         mode='lines', line=dict(dash='dash', width=3), name='Ideal (unitary)'))

for i, Ep_val in enumerate(Ep_vec):
    fig.add_trace(go.Scatter(x=t_list*params["to_time_units"], y=fidelity_curves[i],
                             mode='lines', name=f"Ep={Ep_val*params['to_freq_units']:.1f} MHz",))
fig.update_layout(
    title=f'Protocol fidelity vs time — {PLATFORM.upper()} units',
    xaxis_title=params["time_unit_label"],
    yaxis_title='Fidelity to |1_L⟩',
    template='plotly_white'
)
fig.show()

print("Done: Fidelity curves plotted.")


Ep = 62831853.071795866


/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.



Ep = 125663706.14359173
Ep = 314159265.3589793
Ep = 314159265.3589793
Ep = 628318530.7179586
Ep = 628318530.7179586
Ep = 1256637061.4359171
Ep = 1256637061.4359171
Ep = 3141592653.589793
Ep = 3141592653.589793
Ep = 6283185307.179586
Ep = 6283185307.179586


Done: Fidelity curves plotted.


In [156]:
# =========================
# JFS instantaneous bound vs Ep (MHz) — single cell
# =========================
# ---- helpers (robust scalar & codespace test) ----
def _scalar(x):
    try:
        return complex(x.full()[0,0])    # 1x1 Qobj
    except AttributeError:
        return complex(x)                # already a complex

def in_codespace(psi, tol=1e-6):
    s1 = float(np.real(_scalar(psi.dag()*S1*psi)))
    s2 = float(np.real(_scalar(psi.dag()*S2*psi)))
    return (abs(s1-1.0) < tol) and (abs(s2-1.0) < tol)

# ---- C(t): prefactor without Bose factor (spectral + matrix elements only) ----
def C_out_prefactor_at_t(t, H_func, n_qubits=3, lambda_2=lambda_2):
    Ht = H_func(t)
    evals, evecs = Ht.eigenstates()
    a = evecs[0]; Ea = float(evals[0])

    sm_ops = [tensor([sigmam() if j==i else qeye(2) for j in range(n_qubits)]) for i in range(n_qubits)]
    total = 0.0
    for b_idx in range(1, len(evals)):
        b = evecs[b_idx]
        if in_codespace(b):                 # only C^⊥
            continue
        Eb = float(evals[b_idx])
        omega = Eb - Ea
        if omega <= 1e-12:
            continue
        g_amp = spectral_amp(omega, lambda_2, platform=PLATFORM, model=None)
        if g_amp == 0.0:
            continue
        acc = 0.0
        for sm in sm_ops:
            m = _scalar(a.dag()*sm*b)       # <0|σ^-|b>
            acc += float(np.abs(m)**2)
        total += (g_amp**2) * acc
    return float(total)

# ---- use MAX over time for instantaneous condition ----
def C_max_for_Ep(Ep_val):
    H_fun = make_H_func(Ep_val)
    C_vals = [C_out_prefactor_at_t(t, H_fun, n_qubits=3, lambda_2=lambda_2) for t in t_list]
    return float(np.nanmax(C_vals)) if len(C_vals) else 0.0

# ---- Ep to MHz (x-axis) ----
def Ep_to_MHz(Ep_val):
    # IBM: Ep [rad/s] -> MHz ; QuEra: Ep [rad/µs] -> MHz
    return (Ep_val/(2*np.pi)) / (1e6 if PLATFORM.lower()=="ibm" else 1.0)

# ---- Build instantaneous bound curve vs Ep (MHz) ----
Ep_MHz_axis = []
Smax_bound  = []    # [1/time] = N(Ep) * max_t C(t)

for Ep_val in Ep_vec:
    Cmax = C_max_for_Ep(Ep_val)                                    # max_t C(t)
    N_Ep = bose_einstein_N(Ep_val, T_phys_K)    # Bose at ω = Ep
    Smax_bound.append(N_Ep * Cmax)
    Ep_MHz_axis.append(Ep_to_MHz(Ep_val))

Ep_MHz_axis = np.asarray(Ep_MHz_axis, dtype=float)
Smax_bound  = np.asarray(Smax_bound,  dtype=float)

# ---- Physics-based threshold: ε_R · Ω_max (timescale separation) ----
Omega_max   = omega_max                 # angular coherent scale (rad/s or rad/µs)
epsilon_R   = 1e-2                      # 1% tolerance (adjust if desired)
S_threshold = epsilon_R * Omega_max     # [1/time]

# Find first crossing correctly (avoid NumPy truth-value pitfalls)
mask = Smax_bound <= S_threshold
idx_R = int(np.where(mask)[0][0]) if np.any(mask) else None

# ---- Plot (only instantaneous figure) ----
rate_unit = ('1/s' if PLATFORM.lower()=='ibm' else '1/µs')
ymax = (np.nanmax(Smax_bound) if np.isfinite(Smax_bound).any() else 1.0) * 1.05


fig = go.Figure()

# main curve (note: using your prior conversion factor for consistency with earlier cells)
fig.add_trace(go.Scatter(
    x=Ep_MHz_axis,
    y=Smax_bound * params['to_freq_units'],
    mode='lines+markers',
    name='N(Ep) · max_t C(t)  (upper bound on max_t S_out)'
))

# horizontal threshold line
fig.add_hline(
    y=S_threshold * params['to_freq_units'],
    line=dict(color='green', dash='dash'),
    annotation_text=f"ε_R·Ω_max = {S_threshold*params['to_freq_units']:.3g} [{rate_unit}] (ε_R={epsilon_R:g})",
    annotation_position="top left"
)

# shaded region below the threshold (whole x-range)
x0, x1 = float(np.nanmin(Ep_MHz_axis)), float(np.nanmax(Ep_MHz_axis))
fig.add_shape(
    type="rect",
    x0=x0, x1=x1,
    y0=0, y1=S_threshold * params['to_freq_units'],
    fillcolor="rgba(0, 200, 0, 0.12)",
    line_width=0,
    layer="below"
)

fig.update_layout(
    title=f"Instantaneous leakage bound vs Ep  — {PLATFORM.upper()}, T={T_phys_K} K",
    xaxis_title="Penalty Ep [MHz]",
    yaxis_title=f"N(Ep) · max_t C(t)  [{rate_unit}]",
    template="plotly_white",
    yaxis=dict(range=[0, ymax * params['to_freq_units']])
)
fig.show()

# (Optional) you can still print the smallest Ep that falls under the threshold if you want a numeric statement:
mask = (Smax_bound <= S_threshold)
if np.any(mask):
    first_idx = int(np.where(mask)[0][0])
    print(f"First Ep under threshold (instantaneous): ~{Ep_MHz_axis[first_idx]:.3g} MHz")


First Ep under threshold (instantaneous): ~100 MHz


In [157]:
# --- c_ops_gen_new_with_tracking (robust scalar coefficients per transition) ---

def c_ops_gen_new_with_tracking(t, args):
    """Compute scalar c-coefficients for transitions at time t.

    Returns a dict mapping (a_idx, b_idx) -> [c_coef_scalar, ...] for the instant t.
    The implementation sums rates across single-qubit operators and then takes
    the square root to produce the scalar collapse amplitude, matching the
    convention used in `c_ops_gen_thermal`.
    """
    H_inst_t = args['args_H'](t)
    eigenvalues, eigenstates = H_inst_t.eigenstates()
    c_coef_tracking = {}

    # reference state index and its eigenvalue/vector
    a_idx = 0
    omega_a = float(eigenvalues[a_idx])
    a_state = eigenstates[a_idx]

    for b_offset, b_state in enumerate(eigenstates[1:]):
        b_i = 1 + b_offset
        omega_b = float(eigenvalues[b_i])

        # skip nearly-degenerate pairs
        if np.abs(omega_a - omega_b) < 1e-12:
            continue

        # accumulate total rates for this transition
        rate_total = 0.0

        for i_qubit in range(args['n_qubits']):
            sigmam_i = tensor([sigmam() if j == i_qubit else qeye(2) for j in range(args['n_qubits'])])

            # compute matrix element <b| sigmam_i |a>
            m_obj = b_state.dag() * sigmam_i * a_state
            mag2 = float(np.abs(m_obj) ** 2)
            if mag2 < 1e-8:
                continue

            # spectral and thermal factors (use positive frequency)
            freq = abs(omega_b - omega_a)
            g = spectral_amp(freq, args.get('lambda_2', 0.0))
            N = bose_einstein_N(freq, args.get('T', 0.0))

            rate_abs = N       * (g**2) * mag2
            rate_em  = (N + 1) * (g**2) * mag2
            rate_total += rate_abs + rate_em

        # produce scalar coefficient = sqrt(total_rate)
        c_scalar = float(np.sqrt(max(0.0, rate_total)))

        transition_key = (a_idx, b_i)
        c_coef_tracking.setdefault(transition_key, []).append(c_scalar)

    return c_coef_tracking

# --- End function ---

In [158]:
# --- Collect c-coefs over t_list and plot per-transition (only transitions from state 0) ---
# Define args for tracking (example Ep and parameters)
Ep = params.get('Ep', 0.0)

args_tracking = {
    'args_H': lambda t: make_H_func(Ep)(t),
    'n_qubits': n_qubits,
    'lambda_2': lambda_2,
    'T': T_K
}

c_coef_tracking_all = {}
for t in t_list:
    cct = c_ops_gen_new_with_tracking(t, args_tracking)
    for key, vals in cct.items():
        # consider only transitions that originate from state 0 (i.e., key[0] == 0)
        try:
            a_idx = key[0]
        except Exception:
            # if key isn't indexable, skip
            continue


        # convert to real scalars (abs) and append
        scalars = [float(np.abs(v)) for v in vals]
        c_coef_tracking_all.setdefault(key, []).append(np.mean(scalars) if scalars else 0.0)

if not c_coef_tracking_all:
    print('Warning: no transitions from state 0 were found in the tracking results.')

# Build plot
fig = go.Figure()
for key, series in sorted(c_coef_tracking_all.items()):
    fig.add_trace(go.Scatter(x=t_list*params['to_time_units'], y=series,
                             mode='lines+markers', name=f"{key}"))
fig.update_layout(title='Tracked c-coefficients for transitions from state 0',
                  xaxis_title=params['time_unit_label'], yaxis_title='c_coef (mean abs)', template='plotly_white')
fig.show()

print('Done: c-coef tracking (from state 0) plotted.')

Done: c-coef tracking (from state 0) plotted.


In [159]:
# --- Reduced-grid Ep/T sweep (platform-correct β→Ep mapping, labeled plots) ---

# Use platform temperatures (Kelvin) and beta_vec (dimensionless Ep/(kB T))
T_vec   = temperatures.copy()
beta_ls = beta_vec.copy()

# sample ~10 times from t_list for speed
max_samples = 10
step = max(1, len(t_list) // max_samples)
sampled_t_list = t_list[::step]
if sampled_t_list[-1] != t_list[-1]:
    sampled_t_list = np.concatenate((sampled_t_list, [t_list[-1]]))

# Helper: kB T / ħ in simulator angular units
def kBT_over_hbar_platform(T_K):
    # base: rad/s
    val = (kB * float(T_K)) / hbar
    # convert to rad/µs for QuEra
    return val * (1e-6 if PLATFORM.lower() == "quera" else 1.0)

# Helper: Ep (angular) -> MHz for readability
def Ep_to_MHz(Ep_val):
    return (Ep_val/(2*np.pi)) / (1e6 if PLATFORM.lower() == "ibm" else 1.0)

results_by_beta = {}

for beta in beta_ls:
    sums_per_T = []
    fid_per_T  = []
    Ep_per_T   = []
    EpMHz_per_T= []

    for T in tqdm(T_vec, desc=f"β={beta}", leave=False):
        # ---- physics-correct mapping β → Ep ----
        Ep = beta * kBT_over_hbar_platform(T)     # angular units of the simulator
        Ep_per_T.append(Ep)
        EpMHz_per_T.append(Ep_to_MHz(Ep))

        # build H(t) with this Ep
        H_fun = make_H_func(Ep)

        # args for tracking helper (assumes c_ops_gen_new_with_tracking uses bose_einstein_N with Kelvin)
        args_loc = {
            'args_H': lambda tt, H_fun=H_fun: H_fun(tt),
            'n_qubits': n_qubits,
            'lambda_2': lambda_2,
            'T': float(T)  # Kelvin, consistent with bose_einstein_N
        }

        # 1) accumulate a proxy: sum of |c_coef| over sampled times (dimension: sqrt(rate))
        sum_coefs = 0.0
        try:
            for tt in sampled_t_list:
                cct = c_ops_gen_new_with_tracking(tt, args_loc)
                if isinstance(cct, dict):
                    for key, vals in cct.items():
                        # only transitions starting from ground index 0 (optional filter)
                        try:
                            if key[0] != 0:
                                continue
                        except Exception:
                            pass
                        if isinstance(vals, (list, tuple, np.ndarray)):
                            sum_coefs += float(np.nansum(np.abs(np.asarray(vals, dtype=float))))
                        else:
                            try:
                                sum_coefs += float(abs(vals))
                            except Exception:
                                pass
                elif isinstance(cct, (list, tuple, np.ndarray)):
                    sum_coefs += float(np.nansum(np.abs(np.asarray(cct, dtype=float))))
        except Exception:
            sum_coefs = np.nan

        sums_per_T.append(sum_coefs)

        # 2) piecewise open-system propagation on sampled grid
        rho = logical_zero * logical_zero.dag()
        prev_time = float(sampled_t_list[0])
        final_fid = np.nan
        try:
            for dt in sampled_t_list[1:]:
                t_mid = 0.5 * (prev_time + float(dt))
                try:
                    c_ops_segment = c_ops_gen_thermal(t_mid, H_fun, n_qubits, lambda_2, float(T))
                except Exception:
                    c_ops_segment = None

                t_steps = [prev_time, float(dt)]
                opts = Options(store_states=True, nsteps=10000, rtol=1e-7, atol=1e-9)
                try:
                    res = mesolve(H_fun, rho, t_steps, c_ops=c_ops_segment, args=None,
                                  e_ops=[logical_zero*logical_zero.dag(), logical_one*logical_one.dag()],
                                  options=opts)
                except Exception:
                    # fallback (unitary segment)
                    res = mesolve(H_fun, rho, t_steps, c_ops=None, args=None,
                                  e_ops=[logical_zero*logical_zero.dag(), logical_one*logical_one.dag()],
                                  options=opts)

                prev_time = float(dt)
                rho = res.states[-1]

            final_fid = float((rho1 * rho).tr().real)
        except Exception:
            final_fid = np.nan

        fid_per_T.append(final_fid)

    results_by_beta[beta] = {
        "T": np.array(T_vec, dtype=float),
        "Ep": np.array(Ep_per_T, dtype=float),
        "Ep_MHz": np.array(EpMHz_per_T, dtype=float),
        "sum_coefs": np.array(sums_per_T, dtype=float),
        "fid": np.array(fid_per_T, dtype=float),
    }

# ---- Plot: final fidelity vs summed collapse-coefs (one curve per β), platform-aware labels ----
fig1 = go.Figure()
for beta, data in results_by_beta.items():
    x = data["sum_coefs"]
    y = data["fid"]
    valid = np.isfinite(x) & np.isfinite(y)
    if np.any(valid):
        order = np.argsort(x[valid])
        fig1.add_trace(go.Scatter(
            x=x[valid][order], y=y[valid][order],
            mode='lines+markers',
            name=f"β = Ep/(kB T) = {beta:g}"
        ))
fig1.update_xaxes(title_text='∑ |c_coef| over sampled times (proxy; sqrt(rate)), log scale', type='log')
fig1.update_yaxes(title_text='Final fidelity to |1_L|')
fig1.update_layout(
    title=f'Final fidelity vs summed collapse-coefs (curves = Ep/T families) — {PLATFORM.upper()}',
    template='simple_white'
)
fig1.show()

# ---- Optional: quick table so you see Ep values in MHz for each (β, T) ----
for beta, data in results_by_beta.items():
    print(f"\nβ = {beta}:")
    for T, Ep_val, EpMHz, s, f in zip(data["T"], data["Ep"], data["Ep_MHz"], data["sum_coefs"], data["fid"]):
        unit = "rad/s" if PLATFORM.lower()=="ibm" else "rad/µs"
        print(f"  T={T:.3g} K | Ep={Ep_val:.3e} {unit} (~{EpMHz:.3g} MHz) | sum_coef={s:.3e} | fid={f if np.isfinite(f) else 'nan'}")

# Expose for later
globals()['results_by_beta'] = results_by_beta
print('\nDone: reduced-grid Ep/T sweep complete with platform-correct β→Ep mapping.')


β=0.1:   0%|          | 0/8 [00:00<?, ?it/s]/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.

β=0.1:  12%|█▎        | 1/8 [00:00<00:01,  6.95it/s]/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.

                                                     


β = 0.1:
  T=0.008 K | Ep=1.047e+08 rad/s (~16.7 MHz) | sum_coef=6.415e+03 | fid=0.9297792539730853
  T=0.01 K | Ep=1.309e+08 rad/s (~20.8 MHz) | sum_coef=6.563e+03 | fid=0.9254790461399176
  T=0.012 K | Ep=1.571e+08 rad/s (~25 MHz) | sum_coef=6.673e+03 | fid=0.9221797618149267
  T=0.015 K | Ep=1.964e+08 rad/s (~31.3 MHz) | sum_coef=6.793e+03 | fid=0.9184559360198277
  T=0.02 K | Ep=2.618e+08 rad/s (~41.7 MHz) | sum_coef=6.927e+03 | fid=0.9142181332888326
  T=0.03 K | Ep=3.928e+08 rad/s (~62.5 MHz) | sum_coef=7.076e+03 | fid=0.9093495417735529
  T=0.05 K | Ep=6.546e+08 rad/s (~104 MHz) | sum_coef=7.208e+03 | fid=0.9048978156599747
  T=0.1 K | Ep=1.309e+09 rad/s (~208 MHz) | sum_coef=7.316e+03 | fid=0.9011673645886104

β = 0.2:
  T=0.008 K | Ep=2.095e+08 rad/s (~33.3 MHz) | sum_coef=4.884e+03 | fid=0.9636179150148687
  T=0.01 K | Ep=2.618e+08 rad/s (~41.7 MHz) | sum_coef=4.953e+03 | fid=0.961935187907643
  T=0.012 K | Ep=3.142e+08 rad/s (~50 MHz) | sum_coef=5.003e+03 | fid=0.9607074955

In [160]:
# --- Temperature x Ep sweep (converted snippet) ---
# Uses existing helpers in this notebook: make_H_func, c_ops_gen_thermal, mesolve, initial_state, logical_one, rho1, t_list, e_ops, opts


final_fidelities = {T: [] for T in temperatures}


for T in tqdm(temperatures, desc="Processing temperatures"):
    for Ep in Ep_vec:
        # Build H(t) for this Ep
        H_fun = make_H_func(Ep)  # callable H(t)

        # Initialize state for this run
        rho = initial_state * initial_state.dag()
        prev_time = float(t_list[0])

        # Propagate piecewise across the t_list using collapse operators evaluated at segment midpoints
        for t_next in t_list[1:]:
            t_mid = 0.5 * (prev_time + float(t_next))
            # Compute thermal collapse operators (list of Qobj) at segment midpoint
            try:
                c_ops = c_ops_gen_thermal(t_mid, H_fun, n_qubits, lambda_2, T)
            except Exception:
                c_ops = None

            t_steps = [prev_time, float(t_next)]
            try:
                res = mesolve(H_fun, rho, t_steps, c_ops=c_ops, args=None, e_ops=e_ops, options={'store_states': True})
            except Exception as exc:
                # On solver failure try without collapse operators
                try:
                    res = mesolve(H_fun, rho, t_steps, c_ops=None, args=None, e_ops=e_ops)
                except Exception:
                    raise RuntimeError(f"mesolve failed for Ep={Ep}, T={T}, segment {prev_time}->{t_next}: {exc}")

            # Advance
            rho = res.states[-1]
            prev_time = float(t_next)

        # final fidelity to logical one
        final_fid = float((rho1 * rho).tr().real)
        final_fidelities[T].append(final_fid)

# Plot results
fig = go.Figure()
for T, fidelities in final_fidelities.items():
    fig.add_trace(go.Scatter(x=[i*p["to_freq_units"] for i in Ep_vec], y=fidelities, mode='lines+markers', name=f"T = {T} K"))

fig.update_layout(
    title="Final Fidelity of |1_L> vs Ep",
    xaxis_title="Energy gap parameter (same units as Ep)",
    yaxis_title="Final Fidelity to |1_L>",
    template='plotly_white'
)
fig.show()

# expose results
globals()['final_fidelities_sweep'] = final_fidelities
print('Done: temperature x Ep sweep complete. Results in final_fidelities_sweep')

Processing temperatures: 100%|██████████| 8/8 [00:12<00:00,  1.59s/it]


Done: temperature x Ep sweep complete. Results in final_fidelities_sweep


In [161]:
# --- Find minimum Ep from Ep_vec (notebook) that yields final fidelity >= 0.95 for each temperature ---

# Use notebook-provided grids if available
Ep_values = 2*np.pi*np.linspace(10000, 500000000, 51)
T_values = np.linspace(0.001, 0.1, 21)

required_Ep_for_95 = []

# Ensure logical states exist
if 'logical_zero' not in globals() or 'logical_one' not in globals():
    raise RuntimeError('logical_zero / logical_one not found in the notebook namespace')

# Main sweep: loop over temperatures and Ep_values (Ep_values assumed to be in same units used by make_H_func)
for T in tqdm(T_values, desc='Temperatures'):
    found = False
    last_final_fid = np.nan
    for Ep in tqdm(Ep_values, desc=f'Ep (T={T:.2f})', leave=False):
        H_fun = make_H_func(Ep)

        # initial state for this run
        rho = logical_zero * logical_zero.dag()
        prev_time = float(t_list[0])
        final_res = None
        for t_next in t_list[1:]:
            t_mid = 0.5 * (prev_time + float(t_next))

            # compute thermal collapse operators at midpoint using existing generator
            c_ops_segment = c_ops_gen_thermal(t_mid, H_fun, globals().get('n_qubits', 3), globals().get('lambda_2', 0.1), T)

            t_steps = [prev_time, float(t_next)]
            opts = Options(store_states=True)
            res = mesolve(H_fun, rho, t_steps, c_ops=c_ops_segment, args=None, e_ops=e_ops, options=opts)

            prev_time = float(t_next)
            rho = res.states[-1]
            final_res = res

            if final_res is None:
                final_fidelity = np.nan
            else:
                final_fidelity = float(final_res.expect[1][-1])
            last_final_fid = final_fidelity


        if not np.isnan(last_final_fid) and last_final_fid >= 0.95:
            required_Ep_for_95.append((T, Ep, last_final_fid))
            found = True
            break

    if not found:
        required_Ep_for_95.append((T, None, last_final_fid))

# Unpack and expose results
required_T = [x[0] for x in required_Ep_for_95]
required_Ep = [x[1] for x in required_Ep_for_95]
required_fid = [x[2] for x in required_Ep_for_95]
globals()['required_Ep_for_95'] = required_Ep_for_95
globals()['required_T'] = required_T
globals()['required_Ep'] = required_Ep
globals()['required_fid'] = required_fid

# Print summary
for T, Ep, fid in required_Ep_for_95:
    print(f'T={T:.2f}, required Ep={Ep}, final fidelity={fid if not np.isnan(fid) else "nan"}')

print('Done: required-Ep sweep complete. Results saved to notebook globals: required_Ep_for_95')

Temperatures:   0%|          | 0/21 [00:00<?, ?it/s]/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.

/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.

Temperatures: 100%|██████████| 21/21 [01:44<00:00,  4.99s/it]

T=0.00, required Ep=62893428.28780622, final fidelity=0.9957442601726603
T=0.01, required Ep=125724024.72254065, final fidelity=0.9579626389466323
T=0.01, required Ep=251385217.5920095, final fidelity=0.9556232740895978
T=0.02, required Ep=377046410.46147835, final fidelity=0.9544145598378889
T=0.02, required Ep=502707603.3309472, final fidelity=0.9536982339444644
T=0.03, required Ep=628368796.2004161, final fidelity=0.9532268507063173
T=0.03, required Ep=754029989.0698849, final fidelity=0.9528936635282006
T=0.04, required Ep=879691181.9393538, final fidelity=0.9526458545285679
T=0.04, required Ep=1005352374.8088226, final fidelity=0.9524544024344829
T=0.05, required Ep=1131013567.6782916, final fidelity=0.9523020936283884
T=0.05, required Ep=1256674760.5477602, final fidelity=0.9521780246748913
T=0.06, required Ep=1382335953.4172292, final fidelity=0.952075025923714
T=0.06, required Ep=1507997146.286698, final fidelity=0.9519881544332552
T=0.07, required Ep=1633658339.1561668, final 

In [162]:
# --- Plot required Ep vs T and vs n_th ---
# Verify required results exist
if 'required_T' not in globals() or 'required_Ep' not in globals() or 'required_fid' not in globals():
    raise RuntimeError('required_T/required_Ep/required_fid not found. Run the sweep cell first.')

T = np.array(required_T, dtype=float)
Ep = np.array([np.nan if v is None else v for v in required_Ep], dtype=float)
fid = np.array(required_fid, dtype=float)

if np.all(np.isnan(Ep)):
    print('No Ep found for the fidelity target in the sweep (all values None).')
else:
    # Plot 1: Ep vs Temperature (color = final fidelity)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=T, y=Ep*p["to_freq_units"],  # convert Ep to MHz if needed
        mode='lines+markers',
        line=dict(shape='linear')
    ))
    fig.update_layout(
        title='Required $E_p$ for final fidelity ≥ 0.95 vs Temperature',
        xaxis_title='Temperature T',
        yaxis_title='Required $E_p$',
        template='simple_white',
    )
    fig.show()

In [163]:
# fixed sweep: find minimal Ep (from Ep_vec) that yields final fidelity >= 0.95 and record n_th
Ep_vec = 2*np.pi*np.linspace(10000, 500000000, 51)
T_vec = np.linspace(0.001, 0.1, 21)

required_Ep_for_95 = []

# ensure logical states and helpers are available
assert 'logical_zero' in globals() and 'logical_one' in globals(), "logical states not found"
rho0 = logical_zero * logical_zero.dag()
rho1 = logical_one  * logical_one.dag()
e_ops = [rho0, rho1]

for T in tqdm(T_vec, desc="Temperatures"):
    found = False
    last_final_fid = np.nan
    for Ep in tqdm(Ep_vec, desc=f"Ep (T={T:.3f})", leave=False):
        # build H(t) callable including penalty Ep
        H_fun = make_H_func(Ep)

        # compute avg gap between ground and first excited at start/end
        try:
            H_start = H_fun(float(t_list[0]))
            H_end   = H_fun(float(t_list[-1]))
            eig_start = np.sort(np.real(H_start.eigenenergies()))
            eig_end   = np.sort(np.real(H_end.eigenenergies()))
            gap_start = np.abs(eig_start[1] - eig_start[0])
            gap_end   = np.abs(eig_end[1] - eig_end[0])
            avg_gap = 0.5 * (gap_start + gap_end)
        except Exception:
            avg_gap = np.nan

        # compute n_th using your unit-aware helper
        try:
            n_th = bose_einstein_N(avg_gap, T)
        except Exception:
            n_th = np.nan

        # piecewise propagation using collapse operators at midpoints
        rho = logical_zero * logical_zero.dag()
        prev_time = float(t_list[0])
        solver_failed = False
        for t_next in t_list[1:]:
            t_mid = 0.5 * (prev_time + float(t_next))
            # thermal collapse ops at midpoint
            try:
                c_ops = c_ops_gen_thermal(t_mid, H_fun, n_qubits, lambda_2, T)
            except Exception:
                c_ops = None

            t_steps = [prev_time, float(t_next)]
            opts = Options(store_states=True)
            try:
                res = mesolve(H_fun, rho, t_steps, c_ops=c_ops, args=None, e_ops=e_ops, options=opts)
            except Exception:
                # fallback to no collapse ops
                try:
                    res = mesolve(H_fun, rho, t_steps, c_ops=None, args=None, e_ops=e_ops, options=opts)
                except Exception:
                    solver_failed = True
                    break

            prev_time = float(t_next)
            rho = res.states[-1] if hasattr(res, "states") and res.states else rho
            try:
                last_final_fid = float(res.expect[1][-1])
            except Exception:
                last_final_fid = float((rho1 * rho).tr().real)

        if solver_failed:
            # skip this Ep and continue
            continue

        # record first Ep that meets fidelity threshold
        if not np.isnan(last_final_fid) and last_final_fid >= 0.95:
            required_Ep_for_95.append((T, Ep, n_th, last_final_fid))
            found = True
            break

    if not found:
        required_Ep_for_95.append((T, None, np.nan, last_final_fid))

# expose results
required_T   = [x[0] for x in required_Ep_for_95]
required_Ep  = [x[1] for x in required_Ep_for_95]
required_nth = [x[2] for x in required_Ep_for_95]
required_fid = [x[3] for x in required_Ep_for_95]
globals().update({
    'required_Ep_for_95': required_Ep_for_95,
    'required_T': required_T,
    'required_Ep': required_Ep,
    'required_nth': required_nth,
    'required_fid': required_fid
})

# summary
for T, Ep, nth, fid in required_Ep_for_95:
    print(f"T={T:.3f}, required Ep={Ep}, n_th={nth if not np.isnan(nth) else 'nan'}, final fidelity={fid if not np.isnan(fid) else 'nan'}")

Temperatures:   0%|          | 0/21 [00:00<?, ?it/s]/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.

/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.

Temperatures: 100%|██████████| 21/21 [01:42<00:00,  4.90s/it]

T=0.001, required Ep=62893428.28780622, n_th=0.6196713647504374, final fidelity=0.9957442601726603
T=0.006, required Ep=125724024.72254065, n_th=4.475908049585832, final fidelity=0.9579626389466323
T=0.011, required Ep=251385217.5920095, n_th=8.593936956703464, final fidelity=0.9556232740895978
T=0.016, required Ep=377046410.46147835, n_th=12.716724083272643, final fidelity=0.9544145598378889
T=0.021, required Ep=502707603.3309472, n_th=16.840873788844146, final fidelity=0.9536982339444644
T=0.026, required Ep=628368796.2004161, n_th=20.965600452428458, final fidelity=0.9532268507063173
T=0.031, required Ep=754029989.0698849, n_th=25.090625031550356, final fidelity=0.9528936635282006
T=0.036, required Ep=879691181.9393538, n_th=29.21582344187692, final fidelity=0.9526458545285679
T=0.041, required Ep=1005352374.8088226, n_th=33.341132106952536, final fidelity=0.9524544024344829
T=0.046, required Ep=1131013567.6782916, n_th=37.46651508406414, final fidelity=0.9523020936283884
T=0.051, r

In [164]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=required_nth, y=[i * params['to_freq_units'] for i in required_Ep],
    mode='lines+markers',
    marker=dict(size=8, color=required_T, colorscale='Plasma', colorbar=dict(title='T')),
    line=dict(shape='linear')
))
fig2.update_xaxes(type='log', title_text='Bose–Einstein occupancy n_th')
fig2.update_yaxes(title_text='Required $E_p$')
fig2.update_layout(title='Required $E_p$ vs n_th (mapping via protocol gap)', template='simple_white')

In [178]:
# ==== Dimensionless cumulative leakage with a single y-axis (β·Ep as dashed line) ====
# --- helpers (robust scalar & codespace test) ---
def _scalar(x):
    try:    return complex(x.full()[0,0])
    except: return complex(x)

def in_codespace(psi, tol=1e-6):
    s1 = float(np.real(_scalar(psi.dag()*S1*psi)))
    s2 = float(np.real(_scalar(psi.dag()*S2*psi)))
    return (abs(s1-1.0) < tol) and (abs(s2-1.0) < tol)

# --- total out-of-codespace rate S_out(t) [1/time], per your JFS model ---
def rate_out_of_codespace(t, H_func, n_qubits=3, lambda_2=lambda_2, T_K=T_phys_K):
    Ht = H_func(t)
    evals, evecs = Ht.eigenstates()
    a_idx  = 0
    a_vec  = evecs[a_idx]
    Ea     = float(evals[a_idx])

    sm_ops = [tensor([sigmam() if j==i else qeye(2) for j in range(n_qubits)]) for i in range(n_qubits)]

    S_total = 0.0
    for b_idx in range(len(evals)):
        if b_idx == a_idx:
            continue
        b_vec = evecs[b_idx]
        if in_codespace(b_vec):
            continue
        Eb = float(evals[b_idx])
        omega = abs(Eb - Ea)
        if omega <= 1e-12:
            continue

        rate_abs = 0.0
        rate_em  = 0.0
        for sm in sm_ops:
            m = (b_vec.dag() * sm * a_vec)   # <b|σ^-|a>
            mag2 = float(np.abs(m)**2)
            if mag2 < 1e-16:
                continue
            g  = spectral_amp(omega, lambda_2, platform=PLATFORM, model=None)
            N  = bose_einstein_N(omega, T_phys_K)  # Kelvin
            rate_abs += N       * (g**2) * mag2
            rate_em  += (N + 1) * (g**2) * mag2

        S_total += (rate_abs + rate_em)

    return float(S_total)  # [1/time]

# ---- Build LOCAL H(t) with the Ep you are using (no global mutations) ----
Ep_src = Ep*0.2  # or set Ep_src = your chosen penalty in angular units
H_fun  = make_H_func(Ep_src)

# ---- Compute S_out(t) and its dimensionless cumulative integral ----
S_out_rate = np.array([rate_out_of_codespace(t, H_fun) for t in t_list], dtype=float)  # [1/time]
cum_leak   = np.zeros_like(t_list, dtype=float)
if len(t_list) > 1:
    dt = np.diff(t_list)  # [time]
    # trapezoid rule → dimensionless cumulative leakage
    cum_leak[1:] = np.cumsum(0.5*(S_out_rate[:-1] + S_out_rate[1:]) * dt)

# ---- β·Ep (dimensionless) for the same run on the SAME axis ----
Ep_rad_s = Ep_src * (1e6 if PLATFORM.lower() == "quera" else 1.0)  # convert to rad/s if needed
beta_Ep  = (hbar * Ep_rad_s) / (kB * T_phys_K) if T_phys_K > 0 else np.nan

# ---- Plot (single y-axis, both curves dimensionless) ----
t_plot = t_list * params["to_time_units"]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t_plot, y=cum_leak,
    mode='lines+markers',
    name='Cumulative leakage  ∫ S_out dt'
))
fig.add_trace(go.Scatter(
    x=t_plot, y=[beta_Ep]*len(t_plot),
    mode='lines',
    line=dict(dash='dash', width=2),
    name='β·E_p'
))
fig.update_layout(
    title=f"Dimensionless cumulative leakage vs time — {PLATFORM.upper()}",
    xaxis_title=params["time_unit_label"],
    yaxis_title="Dimensionless (cumulative leakage, β·E_p)",
    template='plotly_white',
    legend=dict(title="")
)
fig.show()

print(f"Final cumulative leakage = {cum_leak[-1]:.3e}; β·E_p = {beta_Ep:.3g} "
      f"(Ep ≈ {(Ep_src/(2*np.pi))/(1e6 if PLATFORM.lower()=='ibm' else 1.0):.3g} MHz, T={T_phys_K} K)")


# ======== Open-system fidelity vs time using c_ops_gen_thermal ========
rho0 = logical_zero * logical_zero.dag()
rho1 = logical_one  * logical_one.dag()
e_ops = [rho0, rho1]

rho    = logical_zero * logical_zero.dag()
prev_t = float(t_list[0])
fidelity_noise = [float((rho1 * rho).tr().real)]  # include t=0

for dt in t_list[1:]:
    t_mid = 0.5 * (prev_t + float(dt))
    # Instantaneous thermal collapse operators at midpoint
    c_ops = c_ops_gen_thermal(t_mid, H_fun, n_qubits=3, lambda_2=lambda_2, T_K=T_K)
    # Propagate one segment
    res = mesolve(H_fun, rho, [prev_t, float(dt)], c_ops=c_ops, e_ops=e_ops,
                  options=Options(store_states=True, nsteps=10000, rtol=1e-7, atol=1e-9))
    rho = res.states[-1]
    prev_t = float(dt)
    fidelity_noise.append(float(res.expect[1][-1]))

# Plot fidelity
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=t_plot,
    y=fidelity_noise,
    mode='lines+markers',
    name='Fidelity to |1_L⟩'
))
fig2.update_layout(
    title=f"Logical |1_L⟩ fidelity over time with thermal noise — {PLATFORM.upper()}",
    xaxis_title=time_label,
    yaxis_title='Fidelity ⟨1_L|ρ|1_L⟩',
    template='plotly_white',
    legend=dict(title="")
)
fig2.show()

print(f"Run params — PLATFORM={PLATFORM}, Ep ≈ {Ep_to_MHz(Ep_val):.3g} MHz, T = {T_K} K, β·Ep ≈ {beta_Ep:.3g}")


Final cumulative leakage = 5.335e-01; β·E_p = 0.25 (Ep ≈ 78 MHz, T=0.015 K)


/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.



Run params — PLATFORM=ibm, Ep ≈ 1.04e+05 MHz, T = 0.015 K, β·Ep ≈ 0.25


In [166]:
# === Cumulative leakage (dimensionless) vs Ep (MHz) ===
# Uses: make_H_func, rate_out_of_codespace (from your previous cell), t_list, Ep_vec, T_phys_K, PLATFORM

Ep_vec = 2*np.pi*np.linspace(1, 500, 21) * 1e6  # example Ep values in angular units
# Convert Ep (angular units) to MHz for x-axis
def Ep_to_MHz(Ep_val):
    # IBM: Ep [rad/s] -> MHz ; QuEra: Ep [rad/µs] -> MHz
    return (Ep_val/(2*np.pi)) / (1e6 if PLATFORM.lower()=="ibm" else 1.0)

def cumulative_leakage_for_Ep(Ep_val, T_K=T_phys_K):
    """Return final (max) dimensionless cumulative leakage for a given Ep."""
    H_fun = make_H_func(Ep_val)  # local Hamiltonian with this Ep
    # instantaneous total leakage rate S_out(t) [1/time]
    S_out = np.array([rate_out_of_codespace(t, H_fun, n_qubits=3, lambda_2=lambda_2, T_K=T_K)
                      for t in t_list], dtype=float)
    if len(t_list) <= 1:
        return 0.0
    # trapezoid integral to get dimensionless cumulative leakage
    dt = np.diff(t_list)
    cum = np.zeros_like(t_list, dtype=float)
    cum[1:] = np.cumsum(0.5*(S_out[:-1] + S_out[1:]) * dt)
    # take the max over time (monotone ⇒ last value)
    return float(np.nanmax(cum))

# Sweep over your platform-specific Ep_vec
Ep_vals     = np.asarray(Ep_vec, dtype=float)
Ep_MHz_axis = np.array([Ep_to_MHz(Ep) for Ep in Ep_vals], dtype=float)
cum_final   = np.array([cumulative_leakage_for_Ep(Ep) for Ep in Ep_vals], dtype=float)

# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Ep_MHz_axis, y=cum_final,
    mode='lines+markers',
    name='final cumulative leakage (∫ S_out dt)'
))
fig.update_layout(
    title=f"Cumulative leakage vs penalty Ep — {PLATFORM.upper()}, T={T_phys_K} K",
    xaxis_title="Penalty Ep [MHz]",
    yaxis_title="Cumulative leakage (dimensionless)",
    template="plotly_white"
)
fig.show()

# Optional: quick print of the smallest Ep meeting a simple tolerance (e.g., ≤ 1%)
tol = 1e-2
mask = np.isfinite(cum_final) & (cum_final <= tol)
if np.any(mask):
    idx = int(np.where(mask)[0][0])
    print(f"First Ep with cumulative leakage ≤ {tol:.1%}: ~{Ep_MHz_axis[idx]:.3g} MHz "
          f"(value={cum_final[idx]:.3g})")
else:
    print("No Ep in Ep_vec reached the example tolerance; consider increasing Ep or relaxing tol.")


No Ep in Ep_vec reached the example tolerance; consider increasing Ep or relaxing tol.


In [167]:
# === Distance Δ(Ep) = β·Ep − max cumulative leakage (only this figure) ===
import numpy as np
import plotly.graph_objects as go

# --- helpers (platform-aware) ---
def Ep_to_MHz(Ep_val):
    # IBM: Ep [rad/s] -> MHz ; QuEra: Ep [rad/µs] -> MHz
    return (Ep_val/(2*np.pi)) / (1e6 if PLATFORM.lower()=="ibm" else 1.0)

def cumulative_leakage_for_Ep(Ep_val, T_K=T_phys_K):
    """Final (max over time) dimensionless cumulative leakage for a given Ep."""
    H_fun = make_H_func(Ep_val)  # local H(t) with this Ep
    S_out = np.array([rate_out_of_codespace(t, H_fun,
                                            n_qubits=3, lambda_2=lambda_2, T_K=T_K)
                      for t in t_list], dtype=float)  # [1/time]
    if len(t_list) <= 1:
        return 0.0
    dt  = np.diff(t_list)
    cum = np.zeros_like(t_list, dtype=float)
    cum[1:] = np.cumsum(0.5*(S_out[:-1] + S_out[1:]) * dt)  # dimensionless
    return float(cum[-1])  # monotone ⇒ final = max

def betaEp_for_Ep(Ep_val, T_K=T_phys_K):
    """β·Ep = ħ Ep / (kB T) (dimensionless); convert Ep to rad/s if QuEra."""
    if T_K <= 0:
        return np.nan
    Ep_rad_s = Ep_val * (1e6 if PLATFORM.lower()=="quera" else 1.0)
    return (hbar * Ep_rad_s) / (kB * T_K)

# --- sweep over Ep_vec ---
Ep_vals      = np.asarray(Ep_vec, dtype=float)
Ep_MHz_axis  = np.array([Ep_to_MHz(Ep)         for Ep in Ep_vals], dtype=float)
cum_final    = np.array([cumulative_leakage_for_Ep(Ep) for Ep in Ep_vals], dtype=float)
betaEp_array = np.array([betaEp_for_Ep(Ep)     for Ep in Ep_vals], dtype=float)

# distance Δ(Ep) (dimensionless)
delta_signed = betaEp_array - cum_final

# --- plot ONLY the distance curve ---
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Ep_MHz_axis, y=delta_signed,
    mode='lines+markers',
    name='Δ(Ep) = β·E_p − max cumulative leakage'
))
fig.add_hline(y=0.0, line=dict(color='gray', dash='dot'))
fig.update_layout(
    title=f"Distance to β·E_p vs penalty — {PLATFORM.upper()}, T={T_phys_K} K",
    xaxis_title="Penalty E_p [MHz]",
    yaxis_title="Δ(Ep) (dimensionless)",
    template="plotly_white",
    legend=dict(title="")
)
fig.show()

# Optional: print the first Ep where leakage ≤ β·Ep
mask = np.isfinite(delta_signed) & (delta_signed >= 0)
if np.any(mask):
    idx = int(np.where(mask)[0][0])
    print(f"First Ep with max cumulative leakage ≤ β·E_p: ~{Ep_MHz_axis[idx]:.3g} MHz "
          f"(β·E_p={betaEp_array[idx]:.3g}, leak={cum_final[idx]:.3g})")
else:
    print("No Ep in Ep_vec satisfied max cumulative leakage ≤ β·E_p.")


First Ep with max cumulative leakage ≤ β·E_p: ~126 MHz (β·E_p=0.402, leak=0.32)


Now - the code for single qubit noise PRX

In [179]:
import numpy as np
from numpy.fft import rfftfreq, irfft
from scipy.interpolate import interp1d
import qutip as qt
from qutip import tensor, sigmax, qeye
import plotly.graph_objects as go

# ---- Expect these to exist from your notebook ----
# PLATFORM, params, omega_t, delta_t, X_L, Z_L, S1, S2, logical_zero, logical_one, t_list, I_L
S_list = [S1, S2]
time_unit_label = params["time_unit_label"]
to_time_units   = params["to_time_units"]
omega_max       = params["omega_max"]

# ============ 1/f noise generator (S0, fmin interface) ============
def generate_1f_noise_coeffs(t_grid,
                             n_qubits=3,
                             n_realizations=1,
                             S0=1e-3,
                             fmin=1e-6,
                             seed=None,
                             normalize=False):
    """
    Synthesize time-domain noise xi[r,q,t] with one-sided PSD S(f)=S0/(2π f) for f>=fmin.
    xi is dimensionless; multiply by noise_strength to get angular units.
    """
    t_grid = np.asarray(t_grid, dtype=float)
    N = int(len(t_grid))
    if N < 2:
        raise ValueError("t_grid must have at least 2 points")
    dt = float(t_grid[1] - t_grid[0])
    freqs = rfftfreq(N, dt)  # in native 1/time units of t_grid
    S = np.zeros_like(freqs, dtype=float)
    mask = freqs >= fmin
    S[mask] = S0 / (2.0 * np.pi * np.maximum(freqs[mask], fmin))
    S[0] = 0.0

    rng = np.random.default_rng(seed)
    xis = np.zeros((int(n_realizations), int(n_qubits), N), dtype=float)
    for r in range(n_realizations):
        for q in range(n_qubits):
            real_part = rng.normal(size=len(freqs))
            imag_part = rng.normal(size=len(freqs))
            spectrum = (real_part + 1j * imag_part) * np.sqrt(np.maximum(S, 0.0) / 2.0)
            spectrum[0] = 0.0
            x = irfft(spectrum, n=N)
            x -= np.mean(x)
            if normalize:
                s = np.std(x)
                if s > 0:
                    x /= s
            xis[r, q, :] = x
    return xis

# ============ Platform defaults for (S0, fmin, noise_strength) ============
def platform_noise_defaults(PLATFORM, T_max, omega_max):
    p = PLATFORM.lower()
    if p == "ibm":
        # seconds & rad/s
        S0 = 1e-3
        fmin = 1.0/(10.0*T_max)          # ~50 kHz for T_max=2 µs
        noise_strength = 2*np.pi*0.25e6  # rad/s (~0.25 MHz ≈ 2% of 12.5 MHz)
    elif p == "quera":
        # microseconds & rad/µs
        S0 = 3e-3
        fmin = 1.0/(10.0*T_max)          # ~0.033 /µs for T_max=3 µs
        noise_strength = 0.15            # rad/µs (~1.5% of 10 rad/µs)
    else:
        raise ValueError("PLATFORM must be 'ibm' or 'quera'")
    return S0, fmin, noise_strength


In [180]:
# --- choose platform-aware values ---
S0, fmin, noise_strength = platform_noise_defaults(PLATFORM, params["T_max"], omega_max)

# --- generate one 1/f noise realization on your native time grid ---
t_grid = np.asarray(t_list, dtype=float)
xis = generate_1f_noise_coeffs(
    t_grid=t_grid,
    n_qubits=3,
    n_realizations=1,
    S0=S0,
    fmin=fmin,
    seed=123,
    normalize=False
)[0]  # shape: (3, N)

# build continuous-time interpolants xi_q(t)
coeffs = [interp1d(t_grid, xis[q], kind='cubic', bounds_error=False,
                   fill_value=(xis[q, 0], xis[q, -1])) for q in range(3)]

# noise operators (X-noise)
X_op, I_op = sigmax(), qeye(2)
noise_ops = [tensor(X_op, I_op, I_op),
             tensor(I_op, X_op, I_op),
             tensor(I_op, I_op, X_op)]

# penalty to use (angular units already platform-correct)
Ep_use = float(params["Ep"])

# ---- Local noisy Hamiltonian (no global Hp/H mutation) ----
def H_noisy(t, args=None):
    H_RAP = X_L * omega_t(t) + Z_L * delta_t(t)
    H_pen = -Ep_use * sum(S_list, 0*I_L)
    H_noise = sum(float(coeffs[q](t)) * noise_strength * noise_ops[q] for q in range(3))
    return (H_RAP + H_pen + H_noise).to('csr')

# ---- Quick diagnostics: effective RMS noise amplitude vs omega_max ----
rms_x = float(np.sqrt(np.mean(xis**2)))
eff_rms_drive = noise_strength * rms_x
print(f"[{PLATFORM}] S0={S0:.2e}, fmin={fmin:.3g} (native units), "
      f"noise_strength={eff_rms_drive*params['to_freq_units']:.3g} MHz "
      f"(~{100*eff_rms_drive/omega_max:.2f}% of ω_max)")

# ---- Evolve (unitary) and plot fidelity to |1_L> ----
res = qt.sesolve(H_noisy, logical_zero, t_list, e_ops=[], args={},
                 options=qt.Options(store_states=True, nsteps=10000, rtol=1e-7, atol=1e-9))
fidelity = [qt.metrics.fidelity(psi, logical_one) for psi in res.states]

t_plot = t_list * to_time_units
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_plot, y=fidelity, mode='lines', name='Fidelity to |1_L⟩'))
fig.update_layout(
    title=(f"Logical |1⟩ fidelity vs time with 1/f X-noise — {PLATFORM.upper()} "
           f"(Ep≈{((Ep_use/(2*np.pi))/(1e6 if PLATFORM.lower()=='ibm' else 1.0)):.3g} MHz)"),
    xaxis_title=time_unit_label,
    yaxis_title='Fidelity',
    template='plotly_white'
)
fig.show()

[ibm] S0=1.00e-03, fmin=5e+04 (native units), noise_strength=2.24e-07 MHz (~0.00% of ω_max)


/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.



trying

In [170]:
# ==== Plot many 1/f noise realizations (per-qubit), with mean and ±1σ ====
import numpy as np
import plotly.graph_objects as go

# --- knobs you can tweak ---
which_qubit = 0            # 0,1,2
R_total     = 200          # total realizations (use 1000 for publication)
R_show      = 20           # number of individual traces to overlay
USE_DRIVE_SCALE = True     # True: plot drive that enters H (scaled); False: plot raw i(t)

# --- get platform defaults, compatible with either version of platform_noise_defaults ---
def _get_platform_defaults():
    try:
        vals = platform_noise_defaults(PLATFORM, params["T_max"], omega_max)  # 3-arg version
    except TypeError:
        vals = platform_noise_defaults(PLATFORM, params["T_max"])            # 2-arg version
    # normalize outputs
    if isinstance(vals, (list, tuple)):
        if len(vals) == 3:
            S0, fmin, default_strength = vals
        elif len(vals) == 2:
            S0, fmin = vals
            default_strength = 2*np.pi*0.25e6 if PLATFORM.lower()=="ibm" else 0.15
        else:
            raise ValueError("platform_noise_defaults returned an unexpected number of values.")
    else:
        raise ValueError("platform_noise_defaults did not return a tuple/list.")
    return S0, fmin, default_strength

S0, fmin, default_strength = _get_platform_defaults()
scale_to_use = (globals().get("noise_strength", default_strength) if USE_DRIVE_SCALE else 1.0)

# --- generate R_total realizations on your simulation grid (expects generate_1f_noise_coeffs to exist) ---
t_grid = np.asarray(t_list, dtype=float)
xis = generate_1f_noise_coeffs(
    t_grid=t_grid,
    n_qubits=3,
    n_realizations=R_total,
    S0=S0,
    fmin=fmin,
    seed=123,
    normalize=False
)  # shape: (R_total, 3, N)

# Convert to plotted amplitude:
#   raw x_q(t) (dimensionless) -> angular drive via scale_to_use -> MHz for readability
xi_drive_MHz = scale_to_use * xis[:, which_qubit, :] * params["to_freq_units"]  # (R_total, N)

# Stats across realizations
mean_trace = np.mean(xi_drive_MHz, axis=0)
std_trace  = np.std(xi_drive_MHz,  axis=0)

t_plot    = t_grid * params["to_time_units"]
time_label = params["time_unit_label"]

# --- Plot ---
fig = go.Figure()

# overlay a subset of realizations (light)
idx_show = np.linspace(0, R_total-1, num=min(R_show, R_total), dtype=int)
for r in idx_show:
    fig.add_trace(go.Scatter(
        x=t_plot, y=xi_drive_MHz[r],
        mode='lines',
        line=dict(color='rgba(0,120,255,0.20)', width=1),
        showlegend=False
    ))

# mean trace
fig.add_trace(go.Scatter(
    x=t_plot, y=mean_trace,
    mode='lines',
    line=dict(color='rgba(0,76,153,1.0)', width=3),
    name='mean noise'
))

# ±1σ band
fig.add_trace(go.Scatter(
    x=np.concatenate([t_plot, t_plot[::-1]]),
    y=np.concatenate([mean_trace-std_trace, (mean_trace+std_trace)[::-1]]),
    fill='toself', fillcolor='rgba(0,120,255,0.15)',
    line=dict(color='rgba(0,0,0,0)'),
    name='±1σ over realizations'
))

title_scale = "drive (scaled)" if USE_DRIVE_SCALE else "raw i(t)"
fig.update_layout(
    title=(f"1/f noise realizations (q{which_qubit}) — {PLATFORM.upper()} [{title_scale}] "
           f"(S0={S0:.1e}, f_min≈{(fmin/1e6 if PLATFORM.lower()=='ibm' else fmin):.3g} MHz, "
           f"scale≈{(scale_to_use*params['to_freq_units']):.3g} MHz)"),
    xaxis_title=time_label,
    yaxis_title="Noise amplitude [MHz]",
    template="plotly_white",
    legend=dict(title="")
)
fig.show()

# quick numeric summary
rms_over_time_per_real = np.sqrt(np.mean(xi_drive_MHz**2, axis=1))
print(f"R_total = {R_total}, qubit = q{which_qubit}")
print(f"Mean RMS over realizations (MHz): {np.mean(rms_over_time_per_real):.3g} "
      f"± {np.std(rms_over_time_per_real):.3g} MHz")


R_total = 200, qubit = q0
Mean RMS over realizations (MHz): 2.38e-07 ± 4.08e-08 MHz


In [171]:
# ==== PSD of 1/f noise across realizations (Welch, mean ± 1σ) ====
import numpy as np
import plotly.graph_objects as go
from scipy.signal import welch

# --- knobs you can tweak ---
which_qubit   = 0      # 0, 1, or 2
R_total       = 200    # number of realizations (use 1000 for publication)
USE_DRIVE_SCALE = True # True: PSD of the drive that enters H; False: raw i(t)

# --- get platform defaults; handle either 2-arg or 3-arg function signature ---
def _get_platform_defaults():
    try:
        vals = platform_noise_defaults(PLATFORM, params["T_max"], omega_max)  # 3-arg version
    except TypeError:
        vals = platform_noise_defaults(PLATFORM, params["T_max"])             # 2-arg version
    if len(vals) == 3:
        S0, fmin, default_strength = vals
    elif len(vals) == 2:
        S0, fmin = vals
        default_strength = 2*np.pi*0.25e6 if PLATFORM.lower()=="ibm" else 0.15
    else:
        raise ValueError("platform_noise_defaults returned unexpected values.")
    return S0, fmin, default_strength

S0, fmin, default_strength = _get_platform_defaults()
scale_to_use = (globals().get("noise_strength", default_strength) if USE_DRIVE_SCALE else 1.0)

# --- generate R_total realizations on your simulation grid ---
t_grid = np.asarray(t_list, dtype=float)
N  = len(t_grid)
dt = float(t_grid[1] - t_grid[0])
fs_native = 1.0 / dt  # Hz (IBM) or 1/µs = MHz (QuEra)

xis = generate_1f_noise_coeffs(
    t_grid=t_grid, n_qubits=3, n_realizations=R_total,
    S0=S0, fmin=fmin, seed=123, normalize=False
)  # shape: (R_total, 3, N)

# convert to the series we want to analyze:
#   raw x_q(t) (dimensionless) -> angular drive via scale_to_use -> MHz for plotting/PSD
amp = scale_to_use * xis[:, which_qubit, :] * params["to_freq_units"]  # (R_total, N), units: MHz

# --- Welch PSD per realization ---
nperseg = max(64, min(1024, (N//4)*2))  # robust segment length
f_native, P0 = welch(amp[0], fs=fs_native, nperseg=nperseg,
                     detrend='constant', scaling='density', return_onesided=True)
P_all = np.empty((R_total, len(f_native)), dtype=float)
P_all[0] = P0
for r in range(1, R_total):
    _, P_all[r] = welch(amp[r], fs=fs_native, nperseg=nperseg,
                        detrend='constant', scaling='density', return_onesided=True)

# --- Convert frequency axis & PSD to per-MHz density ---
if PLATFORM.lower() == "ibm":
    f_MHz = f_native / 1e6                # Hz -> MHz
    P_per_MHz = P_all * 1e6               # (amp^2/Hz) -> (amp^2/MHz)
else:
    f_MHz = f_native                       # native 1/µs == MHz
    P_per_MHz = P_all

mean_psd = np.mean(P_per_MHz, axis=0)
std_psd  = np.std(P_per_MHz, axis=0)

# --- 1/f guide matched at geometric mid-frequency ---
valid = f_MHz > 0
f_mid = np.exp(np.mean(np.log(f_MHz[valid]))) if np.any(valid) else 1.0
def _interp(x, y, x0):
    i = np.searchsorted(x, x0)
    if i <= 0: return y[0]
    if i >= len(x): return y[-1]
    w = (x0 - x[i-1]) / (x[i] - x[i-1])
    return (1-w)*y[i-1] + w*y[i]
P_mid = _interp(f_MHz[valid], mean_psd[valid], f_mid) if np.any(valid) else 1.0
guide = (P_mid * f_mid) / np.maximum(f_MHz, f_MHz[1] if len(f_MHz)>1 else 1.0)

# --- f_min vertical line (in MHz) ---
fmin_MHz = (fmin/1e6) if PLATFORM.lower()=="ibm" else fmin

# --- Plot: mean PSD ±1σ, 1/f guide, f_min line ---
fig = go.Figure()
# ±1σ band
fig.add_trace(go.Scatter(
    x=np.concatenate([f_MHz, f_MHz[::-1]]),
    y=np.concatenate([mean_psd - std_psd, (mean_psd + std_psd)[::-1]]),
    fill='toself', fillcolor='rgba(0,120,255,0.15)',
    line=dict(color='rgba(0,0,0,0)'),
    name='±1σ (over realizations)'
))
# mean PSD
fig.add_trace(go.Scatter(x=f_MHz, y=mean_psd, mode='lines', line=dict(width=3), name='mean PSD'))
# 1/f dashed guide
fig.add_trace(go.Scatter(x=f_MHz, y=guide, mode='lines', line=dict(dash='dash'), name='~1/f guide'))
# f_min vertical line
fig.add_vline(x=fmin_MHz, line=dict(color='gray', dash='dot'),
              annotation_text=f"f_min ≈ {fmin_MHz:.3g} MHz", annotation_position="top right")

fig.update_layout(
    title=f"PSD of 1/f noise (qubit q{which_qubit}) — {PLATFORM.upper()}  (R={R_total})",
    xaxis_title="Frequency [MHz]", yaxis_title="PSD (amplitude² per MHz)",
    xaxis_type="log", yaxis_type="log",
    template="plotly_white", legend=dict(title="")
)
fig.show()

# Optional: slope sanity-check near f_mid
sel = (f_MHz > f_mid/3) & (f_MHz < f_mid*3) & np.isfinite(mean_psd) & (mean_psd > 0)
if np.any(sel):
    slope = np.polyfit(np.log10(f_MHz[sel]), np.log10(mean_psd[sel]), 1)[0]
    print(f"Log–log slope near f≈{f_mid:.3g} MHz: {slope:.2f}  (ideal 1/f → −1)")


/var/folders/sn/yn8dgjs53ml_hdwfm9mp_fyh0000gn/T/ipykernel_22774/956751215.py:46: UserWarning:

nperseg=64 is greater than signal length max(len(x), len(y)) = 51, using nperseg = 51

/var/folders/sn/yn8dgjs53ml_hdwfm9mp_fyh0000gn/T/ipykernel_22774/956751215.py:51: UserWarning:

nperseg=64 is greater than signal length max(len(x), len(y)) = 51, using nperseg = 51



Log–log slope near f≈4.99 MHz: -0.93  (ideal 1/f → −1)


In [172]:
# === Noise-averaged fidelity over R realizations (1/f with S(ω)=S0/ω) ===
import numpy as np
import qutip as qt
from qutip import tensor, sigmax, qeye
from numpy.fft import rfftfreq, irfft
from scipy.interpolate import interp1d
import plotly.graph_objects as go

# --- use your existing helpers/state ---
S_list = [S1, S2]
time_unit_label = params["time_unit_label"]
to_time_units   = params["to_time_units"]

# --- generator you already have (S0,fmin interface) ---
def generate_1f_noise_coeffs(t_grid, n_qubits=3, n_realizations=1, S0=1e-3, fmin=1e-6, seed=None, normalize=False):
    t_grid = np.asarray(t_grid, dtype=float)
    N  = int(len(t_grid))
    dt = float(t_grid[1]-t_grid[0])
    freqs = rfftfreq(N, dt)
    S = np.zeros_like(freqs, dtype=float)
    mask = freqs >= fmin
    S[mask] = S0 / (2.0*np.pi*np.maximum(freqs[mask], fmin))
    S[0] = 0.0
    rng = np.random.default_rng(seed)
    xis = np.zeros((int(n_realizations), int(n_qubits), N), dtype=float)
    for r in range(int(n_realizations)):
        for q in range(int(n_qubits)):
            real = rng.normal(size=len(freqs))
            imag = rng.normal(size=len(freqs))
            spectrum = (real + 1j*imag) * np.sqrt(np.maximum(S,0.0)/2.0)
            spectrum[0] = 0.0
            x = irfft(spectrum, n=N); x -= np.mean(x)
            if normalize:
                s = np.std(x); 
                if s>0: x/=s
            xis[r,q,:] = x
    return xis

# --- platform defaults (exact PRX PSD level; set noise_strength=1 to match i(t) units) ---
def platform_noise_defaults(PLATFORM, T_max):
    if PLATFORM.lower()=="ibm":
        S0    = 1e-3
        fmin  = 1.0/(10.0*T_max)     # ~50 kHz for T_max=2 µs
        scale = 1.0                  # i(t) already in rad/s from PSD
    else:
        S0    = 1e-3
        fmin  = 1.0/(10.0*T_max)     # ~0.033 /µs for T_max=3 µs
        scale = 1.0                  # i(t) in rad/µs
    return S0, fmin, scale

S0, fmin, noise_strength = platform_noise_defaults(PLATFORM, params["T_max"])

# --- theoretical RMS from bandwidth (sanity check) ---
N     = len(t_list)
dt    = float(t_list[1]-t_list[0])
fs    = 1.0/dt             # Hz (IBM) or 1/µs (QuEra)
fmax  = fs/2.0
sigma = np.sqrt((S0/(2.0*np.pi))*np.log(max(fmax/fmin, 1.000001)))  # rad/s or rad/µs
print(f"Implied RMS of i(t): σ ≈ {sigma:.3g} (angular units). In MHz: {sigma*params['to_freq_units']:.3g}.")

# --- Monte-Carlo loop ---
R = 10  # set 1000 for final figure; 200 is faster
Ep_use = float(params["Ep"])
rho0   = logical_zero * logical_zero.dag()
rho1   = logical_one  * logical_one.dag()

fids = np.zeros((R, N), dtype=float)

for r in tqdm(range(R), 'Realizations'):
    # generate per-realization noise traces on the simulation grid
    xi = generate_1f_noise_coeffs(t_list, n_qubits=3, n_realizations=1, S0=S0, fmin=fmin, seed=None, normalize=False)[0]
    # continuous interpolants
    coeffs = [interp1d(t_list, xi[q], kind='cubic', bounds_error=False, fill_value=(xi[q,0], xi[q,-1])) for q in range(3)]

    # build local noisy H(t) for this realization
    X_op, I_op = sigmax(), qeye(2)
    noise_ops = [tensor(X_op, I_op, I_op), tensor(I_op, X_op, I_op), tensor(I_op, I_op, X_op)]

    def H_noisy(t, args=None):
        H_RAP = X_L * omega_t(t) + Z_L * delta_t(t)
        H_pen = -Ep_use * sum(S_list, 0*I_L)
        # i_q(t) already has angular units; do not scale further if you want PRX S(ω)=1e-3/ω literally
        H_noise = sum(float(coeffs[q](t)) * noise_ops[q] for q in range(3))
        return (H_RAP + H_pen + H_noise).to('csr')

    res = qt.sesolve(H_noisy, logical_zero, t_list, e_ops=[rho0, rho1], args={},
                     options=qt.Options(store_states=True, nsteps=10000, rtol=1e-7, atol=1e-9))
    fids[r,:] = np.array(res.expect[1], dtype=float)  # <1_L|ρ|1_L>

# --- average and uncertainty band ---
mean_fid = np.mean(fids, axis=0)
std_fid  = np.std(fids, axis=0)
t_plot   = np.asarray(t_list) * params["to_time_units"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=t_plot, y=mean_fid, mode='lines', name='mean fidelity'))
fig.add_trace(go.Scatter(
    x=np.concatenate([t_plot, t_plot[::-1]]),
    y=np.concatenate([mean_fid-std_fid, (mean_fid+std_fid)[::-1]]),
    fill='toself', fillcolor='rgba(0, 120, 255, 0.15)', line=dict(color='rgba(0,0,0,0)'),
    name='±1σ band'
))
fig.update_layout(
    title=f"Noise-averaged fidelity over {R} realizations — {PLATFORM.upper()} (S0=1e-3, f_min≈{(fmin/1e6 if PLATFORM.lower()=='ibm' else fmin):.3g} MHz)",
    xaxis_title=time_unit_label, yaxis_title='⟨Fidelity to |1_L⟩⟩',
    template='plotly_white', legend=dict(title="")
)
fig.show()


Implied RMS of i(t): σ ≈ 0.0296 (angular units). In MHz: 4.72e-09.


Realizations:   0%|          | 0/10 [00:00<?, ?it/s]/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.

/Users/chenscheim/miniconda3/lib/python3.12/site-packages/qutip/solver/options.py:16: FutureWarning:

Dedicated options class are no longer needed, options should be passed as dict to solvers.

Realizations: 100%|██████████| 10/10 [00:11<00:00,  1.15s/it]


In [173]:
# ==== Noise realizations: many 1/f samples over time (per-qubit view) ====
import numpy as np
import plotly.graph_objects as go

# --- parameters you can tweak ---
which_qubit = 0          # 0, 1, or 2
R_total     = 200        # total realizations to draw (use 1000 for publication-quality)
R_show      = 20         # how many individual traces to overlay (visual clarity)

# Platform-aware defaults for S0, fmin (if not already set elsewhere)
S0, fmin, default_strength = platform_noise_defaults(PLATFORM, params["T_max"], omega_max)

# Choose scaling to plot:
# - If you want PRX-literal i(t) with S(ω)=1e-3/ω (no extra scaling), set scale_to_use = 1.0
# - If you want to show the *drive* that enters H, use your platform's noise_strength (recommended)
scale_to_use = globals().get("noise_strength", default_strength)  # use existing noise_strength if defined

# Generate R_total realizations on your simulation grid
t_grid = np.asarray(t_list, dtype=float)
xis = generate_1f_noise_coeffs(
    t_grid=t_grid,
    n_qubits=3,
    n_realizations=R_total,
    S0=S0,
    fmin=fmin,
    seed=123,
    normalize=False
)  # shape: (R_total, 3, N)

# Convert to the plotted amplitude:
#   raw dimensionless x_q(t)  →  angular drive via scale_to_use  →  MHz via params['to_freq_units']
xi_drive_MHz = scale_to_use * xis[:, which_qubit, :] * params["to_freq_units"]  # shape: (R_total, N)

# Stats across realizations
mean_trace = np.mean(xi_drive_MHz, axis=0)
std_trace  = np.std(xi_drive_MHz,  axis=0)

t_plot = t_grid * params["to_time_units"]
time_label = params["time_unit_label"]

# --- Plot ---
fig = go.Figure()

# light sample traces
idx_show = np.linspace(0, R_total-1, num=min(R_show, R_total), dtype=int)
for r in idx_show:
    fig.add_trace(go.Scatter(
        x=t_plot, y=xi_drive_MHz[r],
        mode='lines',
        line=dict(color='rgba(0,120,255,0.20)', width=1),
        showlegend=False
    ))

# mean trace (bold)
fig.add_trace(go.Scatter(
    x=t_plot, y=mean_trace,
    mode='lines',
    line=dict(color='rgba(0,76,153,1.0)', width=3),
    name='mean noise drive'
))

# ±1σ band
fig.add_trace(go.Scatter(
    x=np.concatenate([t_plot, t_plot[::-1]]),
    y=np.concatenate([mean_trace-std_trace, (mean_trace+std_trace)[::-1]]),
    fill='toself', fillcolor='rgba(0,120,255,0.15)',
    line=dict(color='rgba(0,0,0,0)'),
    name='±1σ over realizations'
))

fig.update_layout(
    title=(f"1/f noise realizations (qubit q{which_qubit}) — {PLATFORM.upper()}  "
           f"(S0={S0:.1e}, f_min≈{(fmin/1e6 if PLATFORM.lower()=='ibm' else fmin):.3g} MHz, "
           f"scale≈{(scale_to_use*params['to_freq_units']):.3g} MHz)"),
    xaxis_title=time_label,
    yaxis_title="Noise drive amplitude [MHz]",
    template="plotly_white",
    legend=dict(title="")
)
fig.show()

# Quick numeric summary
rms_over_time_per_real = np.sqrt(np.mean(xi_drive_MHz**2, axis=1))
print(f"R_total = {R_total}, qubit = q{which_qubit}")
print(f"Mean RMS over realizations (drive, MHz): {np.mean(rms_over_time_per_real):.3g} "
      f"± {np.std(rms_over_time_per_real):.3g} MHz")


TypeError: platform_noise_defaults() takes 2 positional arguments but 3 were given

In [ ]:
# ==== Plot 1/f noise drive vs time (platform-aware units) ====
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import interp1d

# Get platform defaults (S0, fmin, noise_strength) and generate one realization
S0, fmin, noise_strength = platform_noise_defaults(PLATFORM, params["T_max"], omega_max)
xis = generate_1f_noise_coeffs(
    t_grid=np.asarray(t_list, dtype=float),
    n_qubits=3,
    n_realizations=1000,
    S0=S0,
    fmin=fmin,
    seed=123,          # change for a new random draw
    normalize=False
)[0]  # shape: (3, N)

# Evaluate noise on the simulation grid
xi_raw = np.array([xis[q] for q in range(3)], dtype=float)                # dimensionless
xi_drive_ang = noise_strength * xi_raw                                     # angular units (rad/s or rad/µs)
xi_drive_MHz = xi_drive_ang * params["to_freq_units"]                      # convert to MHz for plotting

# Time axis in nice units
t_plot = np.asarray(t_list) * params["to_time_units"]

# Plot: per-qubit noise drive amplitude in MHz
fig = go.Figure()
for q in range(3):
    fig.add_trace(go.Scatter(
        x=t_plot, y=xi_drive_MHz[q], mode='lines', name=f"q{q} noise drive"
    ))
fig.update_layout(
    title=(f"Stochastic 1/f noise drive per qubit — {PLATFORM.upper()} "
           f"(S0={S0:.1e}, f_min≈{(fmin/1e6 if PLATFORM.lower()=='ibm' else fmin):.3g} MHz, "
           f"noise_strength≈{(noise_strength*params['to_freq_units']):.3g} MHz)"),
    xaxis_title=params["time_unit_label"],
    yaxis_title="Noise drive amplitude [MHz]",
    template="plotly_white",
    legend=dict(title="")
)
fig.show()

# Quick numeric summary
rms_dimless = np.sqrt(np.mean(xi_raw**2, axis=1))
rms_drive_MHz = np.sqrt(np.mean(xi_drive_MHz**2, axis=1))
eff_rms_ang = np.sqrt(np.mean((noise_strength*xi_raw)**2))  # angular units
print(f"[{PLATFORM}] S0={S0:.2e}, fmin (native)={fmin:.3g}, "
      f"noise_strength={noise_strength:.3g} (angular)")
print("Per-qubit RMS (dimensionless x):", [f"{v:.3g}" for v in rms_dimless])
print("Per-qubit RMS drive [MHz]:     ", [f"{v:.3g}" for v in rms_drive_MHz])
print(f"Effective RMS / ω_max ≈ {100*eff_rms_ang/omega_max:.2f}%")


[ibm] S0=1.00e-03, fmin (native)=5e+04, noise_strength=1 (angular)
Per-qubit RMS (dimensionless x): ['8.67e-07', '7.38e-07', '1.05e-06']
Per-qubit RMS drive [MHz]:      ['1.38e-13', '1.18e-13', '1.67e-13']
Effective RMS / ω_max ≈ 0.00%
